In [5]:
from __future__ import division, print_function
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [6]:
sess = tf.InteractiveSession() # We need a session for actual values to flow through computation graph

In [7]:
## TODO: We want to multiply 2x3 and 3x1 matrixes x and y, and add a 3x1 vector z
x = np.random.random((2,3))
y = np.random.random((3,1))
z = np.random.random((2,1))

print('x:', x, sep='\n')
print('y:', y, sep='\n')
print('z:', z, sep='\n')

x:
[[ 0.28189587  0.74881406  0.52063016]
 [ 0.181578    0.33071384  0.26811813]]
y:
[[ 0.27780669]
 [ 0.83420453]
 [ 0.71156597]]
z:
[[ 0.05024796]
 [ 0.65874974]]


In [30]:
## numpy Version

final_np = np.mean(np.matmul(x,y) + z)
final_np

18.0

In [31]:
# tensorflow version
final_tf = tf.reduce_mean(tf.matmul(x,y) + z)
final_tf

<tf.Tensor 'Mean_2:0' shape=() dtype=float32>

In [15]:
# To get an actual value, we need to run in a session
print('sess run: ', sess.run(final_tf))
# We can also call a tensor's eval method and pass in a session
print('eval sess:', final_tf.eval(session=sess))
# When using eval with an interactive session, the session argument is optional
print('eval sess:', final_tf.eval())

sess run:  1.14977367365
eval sess: 1.14977367365
eval sess: 1.14977367365


In [16]:
## Batched operations

# Here we will perform batched matrix multiplication
batch_size = 5
N, M, K = 2,3,2
# a_batch is a batch of 5 NxM matrices
# b_batch is a batch of 5 MxK matrices
a_batch = np.random.random((batch_size, N, M))
b_batch = np.random.random((batch_size, M, K))

out = tf.matmul(a_batch,b_batch)
# out is the batch of 5 NxK matrices that result from the batch of products
sess.run(out)

## Is it kind of weird that we need to sess.run to get a constant value?

array([[[ 0.6317564 ,  1.11249802],
        [ 0.40561655,  0.60211636]],

       [[ 0.49942334,  0.51901543],
        [ 0.79068514,  1.0127383 ]],

       [[ 0.43126203,  0.86200233],
        [ 0.24960685,  0.64844905]],

       [[ 0.77804898,  0.63046086],
        [ 0.5625578 ,  0.42793546]],

       [[ 1.42149332,  1.31813935],
        [ 0.07029514,  0.11829601]]])

In [18]:
# In tensorflow, we typically have runtime values in our computation graph
# in some sense, each node in the graph is a function of `Placeholders` that flow into it

# So a function like this is numpy
def matmul_and_add(x,y,z):
    return np.matmul(x,y)+z

# Could be expressed as the following computational graph
N,M,K= 2,3,2
# We'll be multiplying an NxM and MxK matrices and adding a Kx1 vector
x_ph = tf.placeholder(tf.float32, shape=[N,M])
y_ph = tf.placeholder(tf.float32, shape=[M,K])
z_ph = tf.placeholder(tf.float32, shape=[K])
xy = tf.matmul(x_ph,y_ph)
out = xy+z_ph

In [19]:
x = np.random.randint(0,5,[N,M]).astype(np.float32)
y = np.random.randint(0,5,[M,K]).astype(np.float32)
z = np.random.randint(0,5,[K])

print('x:', x, sep='\n')
print('y:', y, sep='\n')
print('z:', z, sep='\n')

tf_out = sess.run(out, feed_dict={x_ph:x, y_ph:y, z_ph:z})
np_out = matmul_and_add(x,y,z)
print("\n\nOUTPUTS \n")
print('np: \n',np_out)
print('tf: \n',tf_out)

x:
[[ 2.  2.  1.]
 [ 1.  4.  3.]]
y:
[[ 2.  2.]
 [ 2.  4.]
 [ 0.  4.]]
z:
[2 2]


OUTPUTS 

np: 
 [[ 10.  18.]
 [ 12.  32.]]
tf: 
 [[ 10.  18.]
 [ 12.  32.]]


In [22]:
# Make sure you feed all required tensors before calling sess.run
tf_out = sess.run(out, feed_dict={x_ph:x, y_ph:y, z_ph:z})

In [28]:
#Notices that basic indexing works
ones = tf.ones(10)
print(sess.run(ones))
print(sess.run(ones[:3]))

# But assignment to tf tensors doesn't work
#ones[:3]=5
# Could be a problem, since we want to change network weights as we learn.

[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
[ 1.  1.  1.]


In [29]:
# Actually not a problem.
# We use variables for Tensors which have values that we wish to change
ones = tf.Variable(tf.ones([10]))
update = ones[:4].assign(np.full(4,5))

# The variables initializer is an operation to set the initial values
# We need to run it before we can use the values of a Variable

sess.run(tf.global_variables_initializer())
ones_val = sess.run(ones)
sess.run(update)
update_val = sess.run(ones)

print('ones val', ones_val)
print('updated val', update_val)

ones val [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
updated val [ 5.  5.  5.  5.  1.  1.  1.  1.  1.  1.]
